In [1]:
from src.learner import *
import time
import pandas as pd

from utilities import remove_cols

words = pd.read_csv('data/words.csv', header=None)
words = words[0].tolist()

# inputs and outputs
X = remove_cols(np.genfromtxt('data/orth.csv', delimiter=","))
Y = remove_cols(np.genfromtxt('data/phon.csv', delimiter=","))


# set seed for generating sample
np.random.seed(982)

target_train_size = 300
train_n = X.shape[0]
test_n = train_n-target_train_size
sample = np.full(train_n, True, dtype=bool)
indices = np.random.choice(train_n, test_n, replace=False)
# Set chosen indices to True because they select the test items not the train items
sample[indices] = False

2024-02-13 21:25:38.832998: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 21:25:38.860208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


This script examines whether or not we can obtain perfect predictions for our sample of 300 target words (randomly selected) by fitting individual models with specific configurations. We will do this before we move on to the brute force tuning in tune_2. The patterns for this model were taken from the `3k` corpus.

Limited search across HPs...

In [15]:
seed = 387

model = learner(X, Y, seed=seed, hidden=100, optimizer=Adam(learning_rate=.1))    
start_time = time.time()                
model.fit(X, Y, epochs=500, batch_size=32, verbose=True)
end_time = time.time()
print(end_time-start_time)

Epoch 1/500
91/91 [==============================] - 0s 745us/step - loss: 0.1865 - binary_accuracy: 0.9389 - mse: 0.0462
Epoch 2/500
91/91 [==============================] - 0s 697us/step - loss: 0.1601 - binary_accuracy: 0.9583 - mse: 0.0332
Epoch 3/500
91/91 [==============================] - 0s 686us/step - loss: 0.2119 - binary_accuracy: 0.9580 - mse: 0.0353
Epoch 4/500
91/91 [==============================] - 0s 641us/step - loss: 0.2459 - binary_accuracy: 0.9602 - mse: 0.0345
Epoch 5/500
91/91 [==============================] - 0s 660us/step - loss: 0.2221 - binary_accuracy: 0.9634 - mse: 0.0313
Epoch 6/500
91/91 [==============================] - 0s 641us/step - loss: 0.2267 - binary_accuracy: 0.9646 - mse: 0.0306
Epoch 7/500
91/91 [==============================] - 0s 629us/step - loss: 0.2211 - binary_accuracy: 0.9652 - mse: 0.0298
Epoch 8/500
91/91 [==============================] - 0s 672us/step - loss: 0.2038 - binary_accuracy: 0.9658 - mse: 0.0286
Epoch 9/500
91/91 [=====

In [19]:
accuracies = batch_accuracy(Y, model.predict(X), dichotomous=True)

91/91 [==============================] - 0s 510us/step


Only a portion of the words are learned perfectly...

In [21]:
len([a for a in accuracies if a == True])

684

In [22]:
correct_words = [word for accuracy, word in zip(accuracies, words) if accuracy == True]

In [25]:
with open('data/correct_words.csv', 'w') as f:
    for word in correct_words:
        f.write('{}\n'.format(word))
f.close()

For examining features that won't learn...

In [17]:
preds = (model.predict(X) > 0.5).astype(int)

91/91 [==============================] - 0s 382us/step


In [18]:
accuracies = preds == Y

In [12]:
np.savetxt("outputs/tune_1_sample_accuracies.csv", accuracies, delimiter=",")

In [13]:
np.savetxt("outputs/tune_1_sample_targets.csv", Y[sample], delimiter=",")